In [25]:
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score
from imblearn.pipeline import Pipeline as ImbPipeline
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,export_graphviz
import graphviz
import dtreeviz
import pickle
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import tree

In [26]:
dataset = pd.read_csv(r"F:\cache_data\frequency_filter\dy\scaler_csv\dy_stander_filter.csv")
dataset.dropna(inplace=True)

In [27]:
# 标签化
dataset["TZ_label"] = dataset.TZ.astype("category").cat.codes
result = dataset.groupby('TZ_label')["TZ"].apply(lambda x: list(x.unique())).to_dict()
print(result)

{0: ['101'], 1: ['102'], 2: ['103'], 3: ['104'], 4: ['105'], 5: ['106'], 6: ['107'], 7: ['108'], 8: ['112'], 9: ['113'], 10: ['114'], 11: ['115'], 12: ['116'], 13: ['117'], 14: ['118'], 15: ['119'], 16: ['11e'], 17: ['120'], 18: ['125'], 19: ['126'], 20: ['129'], 21: ['12b'], 22: ['12c'], 23: ['12d'], 24: ['12e'], 25: ['12f'], 26: ['130'], 27: ['131'], 28: ['132'], 29: ['135'], 30: ['136'], 31: ['137'], 32: ['138'], 33: ['139'], 34: ['13d'], 35: ['13e'], 36: ['13f'], 37: ['141'], 38: ['143'], 39: ['14d'], 40: ['14e'], 41: ['14f'], 42: ['15'], 43: ['15d'], 44: ['15e'], 45: ['15f'], 46: ['18d'], 47: ['1e'], 48: ['1f'], 49: ['20'], 50: ['21'], 51: ['22'], 52: ['23'], 53: ['24'], 54: ['26'], 55: ['29'], 56: ['2b'], 57: ['2f'], 58: ['30'], 59: ['31'], 60: ['32'], 61: ['33'], 62: ['34'], 63: ['4d'], 64: ['4e'], 65: ['4f'], 66: ['60'], 67: ['61d'], 68: ['61e'], 69: ['61f'], 70: ['63'], 71: ['63b'], 72: ['63d'], 73: ['63e'], 74: ['63f'], 75: ['64d'], 76: ['64f'], 77: ['65'], 78: ['65d'], 79: [

In [28]:
dataset['DZ']=dataset['DZ'].astype('category')
dataset['DL']=dataset['DL'].astype('category')
dataset['TZ_label']=dataset['TZ_label'].astype('category')

In [29]:
features = dataset.columns[1:-1]
target = "TZ_label"

In [30]:
X = dataset[features]
y = dataset[target]
# 分割训练集和测试集
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

In [55]:
# 计算每个类别的样本数
class_counts = dataset['TZ_label'].value_counts()

In [56]:
class_counts

TZ_label
39    711
25    671
34    538
24    424
95    335
     ... 
55      2
32      2
66      1
49      1
80      1
Name: count, Length: 102, dtype: int64

In [57]:
# 计算每个类别的样本数
class_counts = dataset['TZ_label'].value_counts()

# 找出样本数少于2的类别
minority_classes = class_counts[class_counts < 6].index.tolist()

In [58]:
minority_classes,len(minority_classes)

([76,
  33,
  100,
  13,
  92,
  42,
  28,
  75,
  96,
  90,
  81,
  67,
  38,
  86,
  16,
  50,
  53,
  91,
  37,
  18,
  55,
  32,
  66,
  49,
  80],
 25)

In [59]:
# 计算每个类别的样本数
class_counts = dataset['TZ_label'].value_counts()

# 找出样本数少于2的类别
minority_classes = class_counts[class_counts < 6].index.tolist()

# 初始化一个空的DataFrame来存储增加的样本
additional_samples = pd.DataFrame(columns=dataset.columns)

# 对每个少数类别重复样本，直到至少有6个样本
for class_label in minority_classes:
    # 获取当前类别的所有样本
    class_samples = dataset[dataset['TZ_label'] == class_label]
    print(class_label)
    # 检查是否存在样本
    if not class_samples.empty:
        # 计算需要增加多少样本
        samples_to_add = 6 - len(class_samples)
        print(class_label)
        # 检查是否需要增加样本
        if samples_to_add > 0:
            # 重复样本
            resampled = resample(class_samples,
                                 replace=True,  # 允许选择相同的样本多次
                                 n_samples=samples_to_add,  # 需要增加的样本数量
                                 random_state=123)  # 随机状态以确保可复现性
            # 将增加的样本添加到additional_samples
            additional_samples = pd.concat([additional_samples, resampled], axis=0)

# 将增加的样本添加回原始数据集
data_augmented = pd.concat([dataset, additional_samples], axis=0).reset_index(drop=True)

# 现在，data_augmented 包含了原始数据和增加的样本，可以用于进一步的分析或模型训练


76
76
33
33
100
100
13
13
92
92
42
42
28
28
75
75
96
96
90
90
81
81
67
67
38
38
86
86
16
16
50
50
53
53
91
91
37
37
18
18
55
55
32
32
66
66
49
49
80
80


C:\Users\Runker\AppData\Local\Temp\ipykernel_19860\22600782.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
C:\Users\Runker\AppData\Local\Temp\ipykernel_19860\22600782.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
C:\Users\Runker\AppData\Local\Temp\ipykernel_19860\22600782.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To 

In [60]:
data_augmented

,TZ,DEM,AnalyticalHillshading,Aspect,ChannelNetworkBaseLevel,ChannelNetworkDistance,ClosedDepressions,ConvergenceIndex,LSFactor,PlanCurvature,...,TMP,VAP,WIND,BIO,LON,LAT,PH,DZ,DL,TZ_label
0,13d,1021.6,0.186124,5.645218,940.00836,81.591610,0.000000,4.197873,13.658210,0.010529,...,15.600000,1.540833,1.683333,195.54823,444445.0,2861300.0,5.476191,9,7,34
1,13d,701.7,1.216353,2.768598,694.73010,6.969910,0.000000,-1.750685,9.647439,-0.005612,...,16.391666,1.587500,1.583333,191.06468,472945.0,2907200.0,6.843954,4,3,34
2,13d,661.0,0.785157,5.674496,663.14667,0.000000,2.146667,1.533146,0.001766,0.000064,...,16.375000,1.588333,1.583333,191.17732,473845.0,2907200.0,6.713590,4,39,34
3,13d,869.8,0.820064,3.988137,857.71760,12.082397,0.000000,2.137851,13.153590,0.005155,...,15.766666,1.555833,1.658333,194.86105,443245.0,2861600.0,5.598476,6,3,34
4,13d,1031.5,0.443475,4.917029,903.36255,128.137450,0.000000,0.071758,14.658850,0.000746,...,15.766666,1.555833,1.658333,194.86105,443245.0,2861900.0,5.441568,3,7,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6072,65f,1062.5,1.437153,2.899373,1033.42090,29.079102,0.000000,2.023754,23.409573,0.007814,...,14.908333,1.464167,1.750000,189.22100,450255.0,2923845.0,5.753367,3,0,80
6073,65f,1062.5,1.437153,2.899373,1033.42090,29.079102,0.000000,2.023754,23.409573,0.007814,...,14.908333,1.464167,1.750000,189.22100,450255.0,2923845.0,5.753367,3,0,80
6074,65f,1062.5,1.437153,2.899373,1033.42090,29.079102,0.000000,2.023754,23.409573,0.007814,...,14.908333,1.464167,1.750000,189.22100,450255.0,2923845.0,5.753367,3,0,80
6075,65f,1062.5,1.437153,2.899373,1033.42090,29.079102,0.000000,2.023754,23.409573,0.007814,...,14.908333,1.464167,1.750000,189.22100,450255.0,2923845.0,5.753367,3,0,80


In [ ]:
# 定义参数网格
params = {
    'decisiontreeclassifier__criterion': ['gini', 'entropy'],
    'decisiontreeclassifier__max_depth': list(range(80, 101, 2)),
    'smote__k_neighbors': [2]  # 设置SMOTE的k_neighbors参数为2
}

# 使用SMOTE处理类别不平衡，这里设置k_neighbors为2
smote = SMOTE(k_neighbors=2)

# 创建决策树分类器实例
dt = DecisionTreeClassifier()

# 创建带有SMOTE的管道
pipeline = ImbPipeline(steps=[('smote', smote), ('decisiontreeclassifier', dt)])

# 使用分层交叉验证
cv = StratifiedKFold(n_splits=5)

# 创建GridSearchCV实例
grid_tree = GridSearchCV(pipeline, params, cv=cv, scoring=make_scorer(accuracy_score))

# 拟合模型
grid_tree.fit(Xtrain, ytrain)

# 打印最佳参数
print(grid_tree.best_params_)
